## ChatGPT

In [1]:
from openai import OpenAI
import json

f = open("API_keys.json")
data = json.load(f)

API_KEY = data["chatgpt"]["api_key"]
f.close()

client = OpenAI(
    # This is the default and can be omitted
    api_key=API_KEY,
)

query = "Hello"

model_id = 'gpt-3.5-turbo'

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": query,
        }
    ],
        model=model_id,
    )

print(response.choices[0].message.content)

Hello! How can I assist you today?
